### This notebook contains the codes to extract and process the necessary simulation data for constructing the machine learning model. The contents must be incorporated to the resilience_metrics.py once finalized.

In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from IPython.display import clear_output

import os
import pandas as pd
from pathlib import Path
from sklearn import metrics
import statistics

import infrarisk.src.network_sim_models.interdependencies as interdependencies
from infrarisk.src.network_sim_models.integrated_network import *

In [4]:

network_dir = Path('../../data/networks/micropolis')
water_folder = network_dir/'water'
power_folder = network_dir/'power'

micropolis_network = IntegratedNetwork(name = 'Micropolis', 
                                       water_folder= water_folder,
                                       power_folder = power_folder,
                                       water_sim_type = 'DDA',
                                       power_sim_type='1ph')

Water network successfully loaded from ..\..\data\networks\micropolis\water/water.inp. The analysis type is set to DDA.
initial simulation duration: 60s; hydraulic time step: 60s; pattern time step: 3600s

Power system successfully loaded from ..\..\data\networks\micropolis\power\power.json. Single phase power flow simulation will be used.



In [5]:
# Set scenarios folder
folder = Path('../../data/networks/micropolis/scenarios')
scenarios = [f for f in sorted(os.listdir(folder))]

#list of recovery strategies to be considered
strategies = ['capacity', 'centrality', 'crewdist', 'zone']

#create the empty dataframe for ML dataset
ml_df = pd.DataFrame(columns =["scenario",
         "strategy",
         'water_perf_ecs',
         'water_perf_pcs',
         'power_perf_ecs',
         'power_perf_pcs',
         'water_mains',
         "water_pumps", 
         "water_tanks",
         "power_lines", 
         "transpo_links",
         "all_compons"])

abnormal_results = []

junc_list = micropolis_network.wn.junction_name_list
base_water_demands = micropolis_network.base_water_node_supply
base_power_demands = micropolis_network.base_power_supply


In [11]:
wn = micropolis_network.wn

In [12]:
for index, scenario in enumerate(scenarios):
    print(index, ". ", scenario)
    ml_df_new = {"scenario": scenario,
         "strategy": None,
         'water_perf_ecs': None,
         'water_perf_pcs': None,
         'power_perf_ecs': None ,
         'power_perf_pcs': None,
         "water_mains": 0, "water_pumps": 0, "water_tanks":0,
         "power_lines": 0, "transpo_links": 0, "all_compons": 0}
    
    disruption_file = pd.read_csv(f"{folder}/{scenario}/disruption_file.csv")
    ml_df_new["all_compons"] = disruption_file.shape[0]
    for _, row in disruption_file.iterrows():
        if row['components'].startswith('P_L'):
            ml_df_new['power_lines'] += 1
        elif row['components'].startswith('W_PMA'):
            ml_df_new['water_mains'] += 1
        elif row['components'].startswith('W_T'):
            ml_df_new['water_tanks'] += 1
        elif row['components'].startswith('W_WP'):
            ml_df_new['water_pumps'] += 1
        elif row['components'].startswith('T_L'):
            ml_df_new['transpo_links'] += 1
        else:
            print("Component type not detectable.")            
    
    for strategy in strategies:  
        ml_df_new['strategy'] = strategy
        water_demands_file = f"{folder}/{scenario}/{strategy}/water_junc_demand.csv"
        power_demands_file = f"{folder}/{scenario}/{strategy}/power_load_demand.csv"
        water_pressure_file = f"{folder}/{scenario}/{strategy}/water_node_pressure.csv"
        
        if os.path.isfile(water_demands_file):
            water_demands = pd.read_csv(water_demands_file)
            water_time_list = water_demands.time/60
            water_time_list = water_time_list.tolist()
            rel_time_list = water_demands['time'] % (24*3600)
            index_list = [int(x/60) for x in rel_time_list]
            water_demands = water_demands[junc_list]
            
            water_pressures = pd.read_csv(water_pressure_file)
            water_pressures = water_pressures[junc_list]
            
            if micropolis_network.water_sim_type == "DDA":
                for i in range(0,water_demands.shape[0]):
                    for j in range(0, water_demands.shape[1]):
                        if water_demands.iloc[i,j] > 0:
                            if water_pressures.iloc[i,j] < 0:
                                water_demands.iat[i,j] = 0
                            elif (0 < water_pressures.iloc[i,j]) & (water_pressures.iloc[i,j] < wn.options.hydraulic.threshold_pressure):
                                water_demands.iat[i,j] = water_demands.iloc[i,j] * math.sqrt(
                                            water_pressures.iloc[i,j] / wn.options.hydraulic.threshold_pressure)
            
            power_demands = pd.read_csv(power_demands_file)
            power_time_list = power_demands.time/60
            power_time_list= power_time_list.tolist()
            
            base_water_demands_new = base_water_demands.iloc[index_list].reset_index(drop=True)
            base_water_demands_new = base_water_demands_new[junc_list]
            
            water_demands_ratio = water_demands/ base_water_demands_new
            water_demands_ratio = water_demands_ratio.clip(upper=1)
            
            water_ecs_list = water_demands_ratio.mean(axis = 1, skipna = True).tolist()
            water_pcs_list = pd.concat([water_demands, base_water_demands_new]).min(level=0).sum(axis=1, skipna = True)/base_water_demands_new.sum(axis=1, skipna = True)
            water_pcs_list = water_pcs_list.tolist()

            base_load_demands = pd.DataFrame(base_power_demands.load.p_mw.tolist() + base_power_demands.motor.pn_mech_mw.tolist()).transpose()
            base_load_demands.columns = base_power_demands.load.name.tolist() + base_power_demands.motor.name.tolist()
            base_load_demands = pd.concat([base_load_demands]*(power_demands.shape[0])).reset_index(drop=True)

            power_demand_ratio = power_demands.iloc[:,1:] / base_load_demands
            power_demand_ratio = power_demand_ratio.clip(upper=1)

            power_ecs_list = power_demand_ratio.mean(axis = 1, skipna = True).tolist()
            power_pcs_list = pd.concat([power_demands.iloc[:,1:], base_load_demands]).min(level=0).sum(axis=1, skipna = True)/base_load_demands.sum(axis=1, skipna = True)
            power_pcs_list = power_pcs_list.tolist()
            
            ml_df_new['water_perf_ecs'] = round(metrics.auc(water_time_list, water_ecs_list), 3)
            ml_df_new['water_perf_pcs'] = round(metrics.auc(water_time_list, water_pcs_list), 3)
            ml_df_new['power_perf_ecs'] = round(metrics.auc(power_time_list, power_ecs_list), 3)
            ml_df_new['power_perf_pcs'] = round(metrics.auc(power_time_list, power_pcs_list), 3)
            
            ml_df = ml_df.append(ml_df_new, ignore_index=True)
            print(ml_df.iloc[-1,:].tolist())
            
        else:
            abnormal_results.append(scenario)
    clear_output(wait=True)
    
#abnormal_results = list(set(abnormal_results))

19 .  track11
['track11', 'capacity', 660.731, 417.153, 3655.65, 3692.606, 7, 0, 0, 5, 3, 15]


In [13]:
ml_df.to_csv("auc_df.csv", index = False)

In [8]:
scenario = 'track18'
strategy = 'capacity'

water_demands_file = f"{folder}/{scenario}/{strategy}/water_junc_demand.csv"
water_pressure_file = f"{folder}/{scenario}/{strategy}/water_node_pressure.csv"
power_demands_file = f"{folder}/{scenario}/{strategy}/power_load_demand.csv"

# if os.path.isfile(water_demands_file):
#     water_demands = pd.read_csv(water_demands_file)
#     water_time_list = water_demands.time/60
#     water_time_list = water_time_list.tolist()
#     rel_time_list = water_demands['time'] % (24*3600)
#     index_list = [int(x/60) for x in rel_time_list]
#     water_demands = water_demands[junc_list]
    
#     power_demands = pd.read_csv(power_demands_file)
#     power_time_list = power_demands.time/60
#     power_time_list= power_time_list.tolist()
    
#     base_water_demands_new = base_water_demands.iloc[index_list].reset_index(drop=True)
#     base_water_demands_new = base_water_demands_new[junc_list]
    
#     water_demands_ratio = water_demands/ base_water_demands_new
#     water_demands_ratio = water_demands_ratio.clip(upper=1)
    
#     water_ecs_list = water_demands_ratio.mean(axis = 1, skipna = True).tolist()
#     water_pcs_list = pd.concat([water_demands, base_water_demands_new]).min(level=0).sum(axis=1, skipna = True)/base_water_demands_new.sum(axis=1, skipna = True)
#     water_pcs_list = water_pcs_list.tolist()

In [9]:

water_demands = pd.read_csv(water_demands_file)
water_pressures = pd.read_csv(water_pressure_file)

water_time_list = water_demands.time/60
water_time_list = water_time_list.tolist()
rel_time_list = water_demands['time'] % (24*3600)
index_list = [int(x/60) for x in rel_time_list]

water_demands = water_demands[junc_list]

In [6]:
water_pressures = water_pressures[junc_list]

for columns in water_demands.columns:
    

# for i in range(0,water_demands.shape[0]):
#     for j in range(0, water_demands.shape[1]):
#         if water_demands.iloc[i,j] > 0:
#             if water_pressures.iloc[i,j] < 0:
#                 water_demands.iat[i,j] = 0
#             elif (0 < water_pressures.iloc[i,j]) & (water_pressures.iloc[i,j] < wn.options.hydraulic.threshold_pressure):
#                 print(i,j, end = ",")
#                 water_demands.iat[i,j] = water_demands.iloc[i,j] * math.sqrt(
#                             water_pressures.iloc[i,j] / wn.options.hydraulic.threshold_pressure)

NameError: name 'water_pressures' is not defined

In [10]:
water_demands.head()

,W_JIN0,W_JTN1,W_JIN2,W_JTN3,W_JIN4,W_JTN5,W_JIN6,W_JTN7,W_JIN8,W_JTN9,...,W_JTN1825,W_JTN1826,W_JTN1827,W_JTN1828,W_JTN1829,W_JTN1830,W_JTN1831,W_JTN1832,W_JTN1833,W_JTN1834
0,0.0,0.000215,0.0,0.000296,0.0,0.000033,0.0,0.000335,0.0,0.000033,...,0.000028,0.000014,0.000021,0.000042,0.000028,0.000014,0.000014,0.000042,0.000028,0.000042
1,0.0,0.000215,0.0,0.000296,0.0,0.000033,0.0,0.000335,0.0,0.000033,...,0.000028,0.000014,0.000021,0.000042,0.000028,0.000014,0.000014,0.000042,0.000028,0.000042
2,0.0,0.000215,0.0,0.000296,0.0,0.000033,0.0,0.000335,0.0,0.000033,...,0.000028,0.000014,0.000021,0.000042,0.000028,0.000014,0.000014,0.000042,0.000028,0.000042
3,0.0,0.000215,0.0,0.000296,0.0,0.000033,0.0,0.000335,0.0,0.000033,...,0.000028,0.000014,0.000021,0.000042,0.000028,0.000014,0.000014,0.000042,0.000028,0.000042
4,0.0,0.000215,0.0,0.000296,0.0,0.000033,0.0,0.000335,0.0,0.000033,...,0.000028,0.000014,0.000021,0.000042,0.000028,0.000014,0.000014,0.000042,0.000028,0.000042


In [15]:
wn = micropolis_network.wn

In [21]:
water_demands.iloc[1,1]

0.0002145066685675

In [ ]:
water_pressures.iloc[:,1]